In [23]:
# Entrenamiento Latitudes y Longitudes
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

#Modelo Tensor
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.regularizers import l2


In [24]:
# Cargamos el Dataset
file_path = "dataset/dataset.csv"
df = pd.read_csv(file_path)

In [25]:
x_columns = [
    'latitude_pos_tx', 'longitude_pos_tx', 'elevation_pos_tx', 'fc', 'elevation', 
    'azimuth', 'year', 'mmdd', 'UTI', 'hour']
x = df[x_columns]
x.loc[:,'fc'] = x['fc']/1E6 
lat_columns = [f'lat_{i}' for i in range(1,101)]
long_columns = [f'long_{i}' for i in range(1,101)]
y_latitudes = df[lat_columns]
y_longitudes = df[long_columns]
#print(df['fc'].head(), x['fc'].head())

In [26]:
x_train, x_test, y_train_lat, y_test_lat, y_train_lon, y_test_lon = train_test_split(
    x, 
    y_latitudes, 
    y_longitudes, 
    test_size=0.2,  
    random_state=42  # Semilla para reproducibilidad
)

In [27]:
l2_reg = 0.001
#Entrada
inputs = Input(shape =(10,))
#Codificación
encoded = Dense(8, activation = 'relu',kernel_regularizer=l2(l2_reg))(inputs)
encoded = Dense(16, activation = "relu",kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(32, activation = "relu",kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(64, activation = "relu",kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(128, activation = "relu",kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(156, activation = "relu",kernel_regularizer=l2(l2_reg))(encoded)
encoded = Dense(180, activation = "relu",kernel_regularizer=l2(l2_reg))(encoded)
# Decodificación separada
latitudes_output = Dense(100, activation = 'linear',kernel_regularizer=l2(l2_reg), name='latitudes_output')(encoded)
longitudes_output = Dense(100, activation='linear', kernel_regularizer=l2(l2_reg), name='longitudes_output')(encoded)
# Definimos el modelo con multiples salidas
autoencoder_Lat_Lon = Model(inputs,outputs=[latitudes_output,longitudes_output])
autoencoder_Lat_Lon.compile(optimizer = 'adamW', loss='mse') #metrics = [custom_mse]
autoencoder_Lat_Lon.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 8)         │         88 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 16)        │        144 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 32)        │        544 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 64)        │      2,112 │ dense_16[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 128)       │      8,320 │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 156)       │     20,124 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 180)       │     28,260 │ dense_19[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latitudes_output    │ (None, 100)       │     18,100 │ dense_20[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ longitudes_output   │ (None, 100)       │     18,100 │ dense_20[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 95,792 (374.19 KB)

 Trainable params: 95,792 (374.19 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
history = autoencoder_Lat_Lon.fit(
    x_train,
    [y_train_lat, y_train_lon],
    epochs=180,
    batch_size=80,
    validation_split=0.2
)

Epoch 1/180
42/42 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - latitudes_output_loss: 843.0312 - longitudes_output_loss: 1443.6842 - loss: 2288.4636 - val_latitudes_output_loss: 14.7751 - val_longitudes_output_loss: 50.5280 - val_loss: 66.6736
Epoch 2/180
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - latitudes_output_loss: 10.6624 - longitudes_output_loss: 47.4687 - loss: 58.6608 - val_latitudes_output_loss: 4.0946 - val_longitudes_output_loss: 33.4594 - val_loss: 38.8322
Epoch 3/180
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - latitudes_output_loss: 3.5429 - longitudes_output_loss: 35.8393 - loss: 39.8303 - val_latitudes_output_loss: 7.9419 - val_longitudes_output_loss: 40.9925 - val_loss: 50.0721
Epoch 4/180
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - latitudes_output_loss: 5.0380 - longitudes_output_loss: 35.4856 - loss: 40.9554 - val_latitudes_output_loss: 2.0594 - val_longitudes_output_loss: 28.0582 - val_loss: 31.2771
Epoch 5/180
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - latitudes_output_loss: 1.9304 

In [29]:
loss = autoencoder_Lat_Lon.evaluate(
    x_test, 
    [y_test_lat, y_test_lon]
)

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - latitudes_output_loss: 0.4729 - longitudes_output_loss: 5.4908 - loss: 6.2766


In [32]:
for layer in autoencoder_Lat_Lon.layers:
    if len(layer.get_weights()) > 0:  # Solo capas con pesos
        weights, biases = layer.get_weights()
        print(f"Pesos de la capa '{layer.name}':\n{weights}")
        print(f"Bias de la capa '{layer.name}':\n{biases}")

# # Ver pesos de una capa en específico
# weights, biases = autoencoder_Lat_Lon.get_layer('dense_1').get_weights()
# print(f"Pesos:\n{weights}")
# print(f"Bias:\n{biases}")



Pesos de la capa 'dense_14':
[[ 1.74362302e-01 -2.90374041e-01 -3.77085119e-01 -5.85336208e-01
  -2.36367669e-34 -8.87282149e-05 -7.17591960e-04 -5.56472671e-33]
 [ 2.91701704e-01 -4.72634822e-01  3.56379688e-01 -2.79029608e-01
   2.25913980e-33 -2.74847122e-03 -4.68937457e-01  3.08518851e-33]
 [-2.77945015e-33  3.28591005e-33 -4.19622978e-33 -5.22495815e-33
   2.12022428e-33 -4.40344225e-33 -5.03160586e-33  3.73547860e-33]
 [-3.92736793e-01  6.56222165e-01  3.50023419e-01 -1.05316766e-01
   3.71113117e-33 -2.13746580e-06 -5.63843846e-01 -2.79161762e-33]
 [ 2.40304582e-02 -8.16731691e-01  1.30406708e-01  8.51635516e-01
   5.31459951e-33 -9.72927960e-10  1.02813494e+00 -1.60015362e-33]
 [-3.43585491e-01  1.88388631e-01  4.32240278e-01  2.36130312e-01
   2.82909734e-33 -3.11737042e-03 -2.18953371e-01  9.64083896e-34]
 [ 5.44190526e-01 -2.69592047e-01  3.22122067e-01 -6.12924732e-02
  -3.11457734e-33 -1.16687547e-02 -5.93812531e-03 -7.62486798e-34]
 [-3.49744797e-01  4.53155905e-01 -4.250

In [33]:
for layer in autoencoder_Lat_Lon.layers:
    if len(layer.get_weights()) > 0:
        weights, biases = layer.get_weights()
        np.save(f"{layer.name}_weights.npy", weights)
        np.save(f"{layer.name}_biases.npy", biases)
        print(f"Pesos y bias de '{layer.name}' guardados.")

Pesos y bias de 'dense_14' guardados.
Pesos y bias de 'dense_15' guardados.
Pesos y bias de 'dense_16' guardados.
Pesos y bias de 'dense_17' guardados.
Pesos y bias de 'dense_18' guardados.
Pesos y bias de 'dense_19' guardados.
Pesos y bias de 'dense_20' guardados.
Pesos y bias de 'latitudes_output' guardados.
Pesos y bias de 'longitudes_output' guardados.


In [34]:
l2_reg = 0.001  # El valor que configuraste
total_penalty = 0

for layer in autoencoder_Lat_Lon.layers:
    if len(layer.get_weights()) > 0:
        weights, _ = layer.get_weights()
        penalty = l2_reg * np.sum(np.square(weights))
        total_penalty += penalty
        print(f"Penalización L2 para la capa '{layer.name}': {penalty}")

print(f"Penalización L2 total: {total_penalty}")

Penalización L2 para la capa 'dense_14': 0.00800652027130127
Penalización L2 para la capa 'dense_15': 0.008981422424316406
Penalización L2 para la capa 'dense_16': 0.00753685474395752
Penalización L2 para la capa 'dense_17': 0.019507490158081054
Penalización L2 para la capa 'dense_18': 0.035599109649658206
Penalización L2 para la capa 'dense_19': 0.053493534088134766
Penalización L2 para la capa 'dense_20': 0.0506007194519043
Penalización L2 para la capa 'latitudes_output': 0.054071044921875
Penalización L2 para la capa 'longitudes_output': 0.06640240478515626
Penalización L2 total: 0.3041991004943848
